In [45]:
! pip install openai python-docx pandas python-dotenv

In [46]:
from pathlib import Path
from docx import Document
import openai
import docx
import pandas as pd
import os
import json
from collections import defaultdict
from dotenv import load_dotenv

In [47]:
load_dotenv()

True

In [48]:
def extract_text_from_docx(docx_path):
    doc = docx.Document(docx_path)
    full_text = []
    for para in doc.paragraphs:
        full_text.append(para.text)
    return '\n'.join(full_text)

In [49]:
from openai import OpenAI
client = OpenAI()

def chat_complete(client, text):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
            "role": "system",
            "content": [
                {
                "type": "text",
                "text": "You are an annotator of transcripts. I want you to rate on a scale of 1-7 how tight being 7 or loose being 1 individuals are describing going about household and childcare tasks. You must return you result in JSON format, for example:\n\n{ eval: { 'tightness': 4,  'sentiment: 1 }\n                    \nHere's a definition and some examples: \n                    \nLoose= they decide to accomplish the household and childcare tasks depending on their schedules and when they have time. If someone observes that a particular task needs to get done, one of them will do it. This tends to be more ad hoc and with little planning or discussion. For example, One person describes a loose approach like this: \"We do not do a formal breakup of how we do things. My partner and I do a lot of things ad hoc. A lot of it is divide and conquer… it's like … I have to go take this one here, can you take this one here?\". Another person describes a loose approach like this: \"It just kind of fell into place. We really didn't talk about it just kind of happened.\" \n\n{'tightness': 1}\n                    \nTight= they decide to accomplish household and childcare tasks according to a predetermined and explicit schedule. The schedule outlines exactly who is responsible for each task, when it is being accomplished during the day/week, and where it will occur. For example, one person describes a tight approach like this: “On Tuesdays and Thursdays when I'm making dinner and picking my son up, my partner just works until dinner, around 6:00. The other two days, Monday and Wednesday, my partner picks my son up, so my partner gets off around 5:00 … and then makes dinner.” Another person describes a tight approach like this: \"I am responsible more for childcare tasks and my partner is more responsible for doing stuff around the house.\" Some terms that indicate tight are \"usually\" and \"always\". \n\n{'tightness': 7}\n\nAlso I want you to give me the sentiment of the transcript overall. Respond as follows: \n\n{ eval: { 'tightness': 1-7,  'sentiment: -1-1}\n\nTranscript: "
                }
            ]
            },
            {
            "role": "user",
            "content": [
                {
                "type": "text",
                "text": text
                }
            ]
            },
        ],
        temperature=0,
        max_tokens=2048,
        top_p=1,
        frequency_penalty=0,
        presence_penalty=0,
        response_format={
            "type": "json_object"
        }
    )
    return response.choices[0].message.content


In [50]:
client = openai.OpenAI()

In [51]:
file_list = list(Path("./data").glob("*.docx"))
file_list.sort()
response_dict = dict()
for file in file_list:
    text = extract_text_from_docx(file)
    print(file.name)
    response_dict[file.name] = chat_complete(client, text)
    

1_1.docx
2_1.docx
3_1.docx


In [52]:
data_dict = defaultdict(list)
for key, value in response_dict.items():
    response_vals = json.loads(value)
    print(response_vals)
    data_dict['file'].append(key)
    data_dict['tightness'].append(response_vals['eval']['tightness'])
    data_dict['sentiment'].append(response_vals['eval']['sentiment'])

print(data_dict)
df = pd.DataFrame(data_dict)
print(df)

{'eval': {'tightness': 2, 'sentiment': 1}}
{'eval': {'tightness': 2, 'sentiment': 0}}
{'eval': {'tightness': 2, 'sentiment': 0}}
defaultdict(<class 'list'>, {'file': ['1_1.docx', '2_1.docx', '3_1.docx'], 'tightness': [2, 2, 2], 'sentiment': [1, 0, 0]})
       file  tightness  sentiment
0  1_1.docx          2          1
1  2_1.docx          2          0
2  3_1.docx          2          0


In [53]:
df.to_csv('output.csv', index=False)